# YOLOv11n Video Inference (Full Features + Color Correction)

**Features:**
- **Batch Processing**: Runs on all videos in input folder.
- **Frame Saving**: Saves `all_frames` and `detected_frames`.
- **Strict NMS**: `agnostic_nms=True`, `iou=0.05` (Ultra Strict).
- **Color Correction**: 
    - If YOLO detects **Green**, but pixel analysis shows **< 5% Green** and **> 5% Red**, it swaps to **Red** (and vice versa).

**Label Format**: `Class Conf | ColorRatio`
**Model**: `yolo11n_mixed/run_9cls_final/weights/best.pt`

In [1]:
import os
import cv2
import shutil
import numpy as np
from ultralytics import YOLO

# --- CONFIGURATION ---
input_video_dir = "/home/prml/StudentsWork/SeokJin/Pprojects/test_video"
output_base_dir = "inference_results_finetune_final_448_half"
model_path = "weights/best_float16_448.tflite"

# Settings
CONF_THRES = 0.298  # Increased from 0.15 to reduce noise
IOU_THRES = 0.05   # Decreased from 0.3 to 0.05 (Ultra Strict NMS to remove nested boxes)
IMGSZ = 448      # Inference image size (matches TFLite)
DEVICE = 0       # GPU Device ID

COLORS = {
    6: (0, 255, 0),    # Green
    7: (0, 0, 255),    # Red
}
DEFAULT_COLOR = (255, 200, 0)

# Load Model
if os.path.exists(model_path):
    print(f"Loading model: {model_path}")
    model = YOLO(model_path)
else:
    print("Model not found! Please check the path.")
    model = None

Loading model: weights/best_float16_448.tflite
WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.


In [2]:
def calculate_color_ratio(roi, cls_id):
    """
    Calculates the ratio of specific color pixels in the ROI.
    7 (Green) -> Green pixels
    8 (Red) -> Red pixels
    """
    if roi.size == 0: return 0.0
    
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    mask = np.zeros(hsv.shape[:2], dtype=np.uint8)
    
    if cls_id == 6: # Green (Wide Range)
        lower = np.array([25, 25, 40])
        upper = np.array([100, 255, 255])
        mask = cv2.inRange(hsv, lower, upper)
        
    elif cls_id == 7: # Red
        lower1 = np.array([0, 50, 50])
        upper1 = np.array([10, 255, 255])
        lower2 = np.array([170, 50, 50])
        upper2 = np.array([180, 255, 255])
        mask = cv2.bitwise_or(
            cv2.inRange(hsv, lower1, upper1),
            cv2.inRange(hsv, lower2, upper2)
        )
        
    else:
        return 0.0
        
    ratio = cv2.countNonZero(mask) / (roi.shape[0] * roi.shape[1])
    return ratio

def process_video_batch():
    if not model or not os.path.exists(input_video_dir):
        return

    video_files = [f for f in os.listdir(input_video_dir) if f.lower().endswith(('.mp4', '.avi', '.mov', '.mkv'))]
    print(f"Found {len(video_files)} videos.")

    for video_file in video_files:
        video_path = os.path.join(input_video_dir, video_file)
        video_name = os.path.splitext(video_file)[0]
        print(f"\nProcessing: {video_file}...")
        
        # Setup Output
        save_dir = os.path.join(output_base_dir, video_name)
        all_frames_dir = os.path.join(save_dir, "all_frames")
        detected_frames_dir = os.path.join(save_dir, "detected_frames")
        
        if os.path.exists(save_dir): shutil.rmtree(save_dir)
        os.makedirs(all_frames_dir, exist_ok=True)
        os.makedirs(detected_frames_dir, exist_ok=True)
        
        cap = cv2.VideoCapture(video_path)
        w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        
        out = cv2.VideoWriter(
            os.path.join(save_dir, f"{video_name}_result.mp4"),
            cv2.VideoWriter_fourcc(*'avc1'), fps, (w, h)
        )
        
        results = model.predict(
            source=video_path, stream=True, 
            conf=CONF_THRES, iou=IOU_THRES, agnostic_nms=True, verbose=False, imgsz=IMGSZ, device=DEVICE
        )
        
        frame_cnt = 0
        detected_cnt = 0
        
        for r in results:
            frame = r.orig_img.copy()
            has_detection = False
            
            if len(r.boxes) > 0:
                has_detection = True
                for box in r.boxes:
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
                    cls_id = int(box.cls[0].item())
                    conf = box.conf[0].item()
                    
                    # Color Correction Logic
                    color_ratio = 0.0
                    if cls_id in [6, 7]:
                        roi = frame[y1:y2, x1:x2]
                        color_ratio = calculate_color_ratio(roi, cls_id)
                        
                        # Correct Weak Detections (Ratio <= 0.05)
                        if color_ratio <= 0.05:
                            other_cls_id = 7 if cls_id == 6 else 6
                            other_ratio = calculate_color_ratio(roi, other_cls_id)
                            
                            # Only swap if opposite color is CLEARLY present
                            if other_ratio > 0.05:
                                cls_id = other_cls_id
                                color_ratio = other_ratio
                                # Note: conf remains same (model confidence)

                    cls_name = model.names[cls_id]

                    # Draw
                    color = COLORS.get(cls_id, DEFAULT_COLOR)
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 3)
                    
                    label = f"{cls_name} {conf:.2f}"
                    if cls_id in [6, 7]:
                        label += f" | P: {color_ratio:.2f}"
                        
                    (tw, th), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)
                    cv2.rectangle(frame, (x1, y1 - 20), (x1 + tw, y1), color, -1)
                    cv2.putText(frame, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
            
            out.write(frame)
            
            # Frame Saving
            fname = f"frame_{frame_cnt:05d}.jpg"
            cv2.imwrite(os.path.join(all_frames_dir, fname), frame)
            if has_detection:
                cv2.imwrite(os.path.join(detected_frames_dir, fname), frame)
                detected_cnt += 1
            
            frame_cnt += 1
            if frame_cnt % 50 == 0:
                print(f"  Frame {frame_cnt}...", end='\r')
                
        cap.release()
        out.release()
        print(f"\nSaved: {video_name} (Detected: {detected_cnt}/{frame_cnt})")

process_video_batch()

Found 6 videos.

Processing: 20251209_221018.mp4...


E0000 00:00:1765522184.166333  524388 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765522184.172460  524388 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765522184.188396  524388 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765522184.188419  524388 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765522184.188421  524388 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765522184.188423  524388 computation_placer.cc:177] computation placer already registered. Please check linka

Loading weights/best_float16_448.tflite for TensorFlow Lite inference...


/home/prml/miniconda3/envs/rfdetr/lib/python3.11/site-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


  Frame 5300...
Saved: 20251209_221018 (Detected: 4784/5311)

Processing: 20251209_220841.mp4...
  Frame 300...
Saved: 20251209_220841 (Detected: 194/343)

Processing: 20251209_220956.mp4...
  Frame 550...
Saved: 20251209_220956 (Detected: 341/568)

Processing: 20251209_220543.mp4...
  Frame 1950...
Saved: 20251209_220543 (Detected: 1632/1974)

Processing: 20251209_220653.mp4...
  Frame 3000...
Saved: 20251209_220653 (Detected: 2029/3028)

Processing: 20251209_221649.mp4...
  Frame 5200...
Saved: 20251209_221649 (Detected: 4529/5205)
